In [17]:
import joblib
import numpy as np
from catboost import CatBoostClassifier, Pool, cv
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
import imblearn


class Model:
    cat_features = [2,3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
    catBoost = None
    n_estimators = None
    criterion = 'gini'
    max_depth = None
    max_samples = None
    n_jobs = -1
    mappings = {}
    objectType = None
    boolType = None
    def __init__(self):
        self.catBoost = CatBoostClassifier(iterations=3000,
                           depth=2,
                          # learning_rate=1,
                           loss_function='MultiClass',
                           # custom_loss = ["AUC","Accuracy"],
                            train_dir="MultiClass",
                            verbose=True)

    def train(self, train_data, eval_data):
        self.catBoost.fit(train_data,eval_set = eval_data,plot=True)

    def predict(self, test_data):
        y_pred = self.catBoost.predict(test_data)
        return y_pred
    def load(self):
        self.catBoost = joblib.load("./random_forest.joblib")

    def save(self,path):
        joblib.dump(self.catBoost, path)

    def loadData(self):
        import pandas as pd
      #  excel_data_df = pandas.read_csv('mainDataset.csv', on_bad_lines='skip', delimiter=";")
        """
        data = np.genfromtxt("NEWDATA4.csv",delimiter=';',encoding = 'UTF-8', dtype=None)
        data = data[1:,::]
        datax = np.delete(data, 7, 1)[::, ::]
        datay = data[::, 7]
        """
        df = pd.read_csv('ActualDatasets.csv', skipinitialspace=True, sep=';')
        return df
    def preprocess(data):
        import nltk
        import sklearn

        data = data.drop("WordsCount", axis = 1)
        data = data.drop("LeftIndentation", axis = 1)
        data = data.drop("SpecialIndentation", axis = 1)
        data = data.drop("SpaceBefore", axis = 1)
        data = data.drop("SpaceAfter", axis = 1)
        data = data.drop("RightIndentation", axis = 1)
        data = data.drop("LineSpacing", axis = 1)
        data = data.drop("Alignment", axis = 1)
        target_names = list(data['CurElementMark'].unique())
        data.info()
        objectType = list(data.select_dtypes(['object']).columns)
        boolType = list(data.select_dtypes(['bool']).columns)
        return data
        
    def mapping(data, boolType, objectType):
        from sklearn import preprocessing
        from sklearn.preprocessing import LabelEncoder
        le = preprocessing.LabelEncoder()
        for i in boolType:
            data[i] = data[i].astype(int)
        for i in objectType:
            data[i] = data[i].fillna('0')
            le.fit(data[i])
            numpy.save(i+'.npy', le.classes_)
            data[i] = le.transform(data[i])
            #df[i] = df.apply(lambda i:le.transform(df[i].astype(str)), axis=0, result_type='expand')
            mappings[i] = dict(zip(le.classes_, le.transform(le.classes_)))
        return mappings

        
    def sampling(datax,datay):
        oversample = RandomOverSampler(sampling_strategy='minority')
        X_over, y_over = oversample.fit_resample(datax, datay)
        X_over, y_over = oversample.fit_resample(X_over, y_over)
        X_over, y_over = oversample.fit_resample(X_over, y_over)
        undersample = RandomUnderSampler(sampling_strategy= 'majority')
        X_over, y_over = undersample.fit_resample(X_over, y_over)
        return train_test_split(X_over,y_over,test_size=0.35, random_state = 1)
    
    def prepare_data(data_train_x,data_train_y,data_test_x,data_test_y):
        '''
        train_y = train[::, 8]
        train_x = np.delete(train, 8, 1)[::, ::]
        train_y = train_y.reshape(len(train_y))
        test_y = test[::, 8]
        test_y = test_y.reshape(len(test_y))
        test_x = np.delete(test, 8, 1)[::, ::]
        '''


        train_dataset = Pool(data=data_train_x,
                             label=data_train_y,
                             cat_features=cat_features)

        eval_dataset = Pool(data=data_test_x,
                            label=data_test_y,
                            cat_features=cat_features)
        return train_dataset,eval_dataset,train_y

model = Model()
data = model.loadData()
data = Model.preprocess(data)
mappings = Model.mapping(data, model.boolType, model.objectType);
mappings
"""
model = Model()
train_x,test_x,train_y,test_y = model.loadData()
train_dataset,eval_dataset, train_y = Model.prepare_data(train_x,train_y,test_x,test_y)
model.train(train_dataset,eval_dataset)
import time
a = time.perf_counter()
y_pred = model.predict(eval_dataset)
a = time.perf_counter() - a
print(a)
from sklearn.metrics import confusion_matrix
CM = confusion_matrix(y_pred, test_y)
print(CM)
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2219 entries, 0 to 2218
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Content                              2219 non-null   object
 1   SpecialSymbolsCount                  2219 non-null   int64 
 2   SymbolCount                          2219 non-null   int64 
 3   Lowercase                            2219 non-null   bool  
 4   Uppercase                            2219 non-null   bool  
 5   LastSymbolPd                         1558 non-null   object
 6   FirstKeyWord                         518 non-null    object
 7   PrevElementMark                      2219 non-null   object
 8   CurElementMark                       2219 non-null   object
 9   NextElementMark                      2219 non-null   object
 10  FullBold                             2219 non-null   bool  
 11  FullItalic                           2219 n

TypeError: 'NoneType' object is not iterable

In [3]:
from sklearn.metrics import classification_report
print(classification_report(y_pred, test_y))

              precision    recall  f1-score   support

          b1       0.71      1.00      0.83        40
          b2       0.95      0.88      0.91       158
          c1       0.88      0.78      0.83        49
          c2       0.92      0.91      0.91       118
          d2       0.93      0.98      0.96       287
          f1       0.98      1.00      0.99        41
          f3       1.00      1.00      1.00       911
          g1       0.99      1.00      0.99        67
          h1       1.00      1.00      1.00       198
          i1       1.00      0.99      1.00       934
          j0       1.00      0.99      1.00       924

    accuracy                           0.98      3727
   macro avg       0.94      0.96      0.95      3727
weighted avg       0.99      0.98      0.98      3727



In [9]:
import joblib
import numpy as np
from catboost import CatBoostClassifier, Pool, cv
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
import imblearn


class Model:
    catBoost = None
    n_estimators = None
    criterion = 'gini'
    max_depth = None
    max_samples = None
    n_jobs = -1
    def __init__(self):
        self.catBoost = CatBoostClassifier(iterations=3000,
                           depth=2,
                          # learning_rate=1,
                           loss_function='MultiClass',
                           # custom_loss = ["AUC","Accuracy"],
                            train_dir="MultiClass",
                            verbose=True)

    def train(self, train_data, eval_data):
        self.catBoost.fit(train_data,eval_set = eval_data,plot=True)

    def predict(self, test_data):
        y_pred = self.catBoost.predict(test_data)
        return y_pred
    def load(self):
        self.catBoost = joblib.load("./random_forest.joblib")

    def save(self,path):
        joblib.dump(self.catBoost, path)

    def loadData(self):
        import pandas
      #  excel_data_df = pandas.read_csv('mainDataset.csv', on_bad_lines='skip', delimiter=";")
        data = np.genfromtxt("СтарыйДатасет.csv",delimiter=';',encoding = 'UTF-8', dtype=None)
        data = data[1:,::]
        datax = np.delete(data, 8, 1)[::, ::]
        datay = data[::, 8]
        oversample = RandomOverSampler(sampling_strategy='minority')
        X_over, y_over = oversample.fit_resample(datax, datay)
        X_over, y_over = oversample.fit_resample(X_over, y_over)
        X_over, y_over = oversample.fit_resample(X_over, y_over)
        undersample = RandomUnderSampler(sampling_strategy= 'majority')
        X_over, y_over = undersample.fit_resample(X_over, y_over)
        return train_test_split(X_over,y_over,test_size=0.35, random_state = 1)
    def prepare_data(data_train_x,data_train_y,data_test_x,data_test_y):
        '''
        train_y = train[::, 8]
        train_x = np.delete(train, 8, 1)[::, ::]
        train_y = train_y.reshape(len(train_y))
        test_y = test[::, 8]
        test_y = test_y.reshape(len(test_y))
        test_x = np.delete(test, 8, 1)[::, ::]
        '''
        cat_features = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,16,17,18]

        train_dataset = Pool(data=data_train_x,
                             label=data_train_y,
                             cat_features=cat_features)

        eval_dataset = Pool(data=data_test_x,
                            label=data_test_y,
                            cat_features=cat_features)
        return train_dataset,eval_dataset,train_y

model = Model()
train_x,test_x,train_y,test_y = model.loadData()
train_dataset,eval_dataset, train_y = Model.prepare_data(train_x,train_y,test_x,test_y)
model.train(train_dataset,eval_dataset)
import time
a = time.perf_counter()
y_pred = model.predict(eval_dataset)
a = time.perf_counter() - a

from sklearn.metrics import confusion_matrix
CM = confusion_matrix(y_pred, test_y)
print(CM)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.072498
0:	learn: 2.0029563	test: 2.0108316	best: 2.0108316 (0)	total: 166ms	remaining: 8m 17s
1:	learn: 1.7234227	test: 1.7338662	best: 1.7338662 (1)	total: 187ms	remaining: 4m 40s
2:	learn: 1.5389617	test: 1.5513236	best: 1.5513236 (2)	total: 209ms	remaining: 3m 28s
3:	learn: 1.3936136	test: 1.4071436	best: 1.4071436 (3)	total: 231ms	remaining: 2m 52s
4:	learn: 1.2620420	test: 1.2738284	best: 1.2738284 (4)	total: 249ms	remaining: 2m 28s
5:	learn: 1.1440995	test: 1.1519462	best: 1.1519462 (5)	total: 269ms	remaining: 2m 14s
6:	learn: 1.0467235	test: 1.0524653	best: 1.0524653 (6)	total: 288ms	remaining: 2m 3s
7:	learn: 0.9681247	test: 0.9730267	best: 0.9730267 (7)	total: 307ms	remaining: 1m 54s
8:	learn: 0.9001554	test: 0.9036107	best: 0.9036107 (8)	total: 325ms	remaining: 1m 47s
9:	learn: 0.8408338	test: 0.8442395	best: 0.8442395 (9)	total: 343ms	remaining: 1m 42s
10:	learn: 0.7861197	test: 0.7891480	best: 0.7891480 (10)	total: 362ms	remaining: 1m 38s
11:	learn: 0

In [11]:
print(a)

0.005345399957150221
